## Detecting Payment Card Fraud

- we'll look at a credit card fraud detection dataset and build a binary classification model that can identify transactions as either fraudulent or valid based on provided *historical* data. 

## Labeled Data

- The payment fraud data set (Dal Pozzolo et al. 2015) was downloaded from [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud/data).
- This has features and labels for thousands of credit card transactions, each of which is labeled as fraudulent or valid. 
- In this notebook, we'd like to train a model based on the features of these transactions so that we can predict risky or fraudulent transactions in the future.

## Binary Classification

- Since we have true labels to aim for, we'll take a **supervised learning** approach and train a binary classifier to sort data one of our two transaction classes: fraudulent or valid.
- We'll train a model on training data and see how well it generalizes on some test data.

The notebook will be broken down into a few steps:
* Loading and exploring the data
* Splitting the data into train/test sets
* Defining and training a LinearLearner, binary classifier
* Making improvements on the model
* Evaluating and comparing model test performance

## Making Improvements

- A lot of this notebook will focus on making improvements, as discussed in [this SageMaker blog post](https://aws.amazon.com/blogs/machine-learning/train-faster-more-flexible-models-with-amazon-sagemaker-linear-learner/). Specifically, we'll address techniques for:

1. **Tuning a model's hyperparameters** and aiming for a specific metric, such as high recall or precision.
2. **Managing class imbalance**, which is when we have many more training examples in one class than another (in this case, many more valid transactions than fraudulent).

In [1]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

%matplotlib inline

In [2]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

## Loading and Exploring the Data

In [ ]:
!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
!unzip creditcardfraud

In [3]:
local_data = 'creditcard.csv'

transaction_df = pd.read_csv(local_data)
print('Data shape (rows, cols): ', transaction_df.shape)

transaction_df.head()

Data shape (rows, cols):  (284807, 31)



,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## Calculate the percentage of fraudulent data

In [4]:
def fraudulent_percentage(transaction_df):
    '''Calculate the fraction of all data points that have a 'Class' label of 1; fraudulent.
       :param transaction_df: Dataframe of all transaction data points; has a column 'Class'
       :return: A fractional percentage of fraudulent data points/all points
    '''
    counts = transaction_df['Class'].value_counts()
    
    fraud_cnts = counts[1]
    valid_cnts = counts[0]
    
    fraud_percentage = fraud_cnts/(fraud_cnts+valid_cnts)
    
    return fraud_percentage

In [7]:
fraud_percentage = fraudulent_percentage(transaction_df)

print('Fraudulent percentage = ', fraud_percentage)
print('Total # of fraudulent pts: ', fraud_percentage*transaction_df.shape[0])
print('Out of (total) pts: ', transaction_df.shape[0])

Fraudulent percentage =  0.001727485630620034
Total # of fraudulent pts:  492.0
Out of (total) pts:  284807


## Split into train/test datasets

In [8]:
def train_test_split(transaction_df, train_frac= 0.7, seed=1):
    '''Shuffle the data and randomly split into train and test sets;
       separate the class labels (the column in transaction_df) from the features.
       :param df: Dataframe of all credit card transaction data
       :param train_frac: The decimal fraction of data that should be training data
       :param seed: Random seed for shuffling and reproducibility, default = 1
       :return: Two tuples (in order): (train_features, train_labels), (test_features, test_labels)
       '''
    
    df_matrix = transaction_df.values
    np.random.seed(seed)
    np.random.shuffle(df_matrix)
    
    train_size = int(df_matrix.shape[0] * train_frac)    
    train_features  = df_matrix[:train_size, :-1]    
    train_labels = df_matrix[:train_size, -1]
    
    test_features = df_matrix[train_size:, :-1]
    test_labels = df_matrix[train_size:, -1]
    
    return (train_features, train_labels), (test_features, test_labels)

In [9]:
(train_features, train_labels), (test_features, test_labels) = train_test_split(transaction_df, train_frac=0.7)

In [10]:
print('Training data pts: ', len(train_features))
print('Test data pts: ', len(test_features))
print()

print('First item: \n', train_features[0])
print('Label: ', train_labels[0])
print()

assert len(train_features) > 2.333*len(test_features), \
        'Unexpected number of train/test points for a train_frac=0.7'
assert np.all(train_labels)== 0 or np.all(train_labels)== 1, \
        'Train labels should be 0s or 1s.'
assert np.all(test_labels)== 0 or np.all(test_labels)== 1, \
        'Test labels should be 0s or 1s.'

print('Tests passed!')

Training data pts:  199364
Test data pts:  85443

First item: 
 [ 1.19907000e+05 -6.11711999e-01 -7.69705324e-01 -1.49759145e-01
 -2.24876503e-01  2.02857736e+00 -2.01988711e+00  2.92491387e-01
 -5.23020325e-01  3.58468461e-01  7.00499612e-02 -8.54022784e-01
  5.47347360e-01  6.16448382e-01 -1.01785018e-01 -6.08491804e-01
 -2.88559430e-01 -6.06199260e-01 -9.00745518e-01 -2.01311157e-01
 -1.96039343e-01 -7.52077614e-02  4.55360454e-02  3.80739375e-01
  2.34403159e-02 -2.22068576e+00 -2.01145578e-01  6.65013699e-02
  2.21179560e-01  1.79000000e+00]
Label:  0.0

Tests passed!


## Create a LinearLearner Estimator

In [11]:
from sagemaker import LinearLearner

prefix = 'creditcard'

output_path = 's3://{}/{}'.format(bucket, prefix)
output_path

's3://sagemaker-us-west-2-976575622760/creditcard'

## Define LinearLearner

In [12]:
linear = LinearLearner(role=role,
                       train_instance_count=1, 
                       train_instance_type='ml.c4.xlarge',
                       predictor_type='binary_classifier', 
                       output_path=output_path,
                       sagemaker_session=sagemaker_session,
                       epochs=15) 

## Convert data

In [13]:
train_x_np = train_features.astype('float32')
train_y_np = train_labels.astype('float32')

formatted_train_data = linear.record_set(train_x_np, labels=train_y_np)

##  Train the Estimator

In [14]:
%%time 
linear.fit(formatted_train_data)

2019-08-20 04:20:12 Starting - Starting the training job...
2019-08-20 04:20:17 Starting - Launching requested ML instances......
2019-08-20 04:21:22 Starting - Preparing the instances for training...
2019-08-20 04:22:04 Downloading - Downloading input data......
2019-08-20 04:23:00 Training - Training image download completed. Training in progress.
Docker entrypoint called with argument(s): train
[08/20/2019 04:23:03 INFO 140429891430208] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u

[2019-08-20 04:23:22.039] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 5995, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.010330928617985404, "sum": 0.010330928617985404, "min": 0.010330928617985404}}, "EndTime": 1566275002.039478, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1566275002.039374}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.008765228896883864, "sum": 0.008765228896883864, "min": 0.008765228896883864}}, "EndTime": 1566275002.039608, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1566275002.039583}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0103271253791

[2019-08-20 04:23:27.842] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 5797, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.008034130657138537, "sum": 0.008034130657138537, "min": 0.008034130657138537}}, "EndTime": 1566275007.842617, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1566275007.842524}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006699231958868516, "sum": 0.006699231958868516, "min": 0.006699231958868516}}, "EndTime": 1566275007.842702, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1566275007.842682}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0080317853203

[2019-08-20 04:23:40.173] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 13, "duration": 6040, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006276343655945668, "sum": 0.006276343655945668, "min": 0.006276343655945668}}, "EndTime": 1566275020.174149, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1566275020.174025}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005225318925464573, "sum": 0.005225318925464573, "min": 0.005225318925464573}}, "EndTime": 1566275020.174275, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1566275020.174253}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006276083052

[2019-08-20 04:23:52.453] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 17, "duration": 6144, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005591161470317361, "sum": 0.005591161470317361, "min": 0.005591161470317361}}, "EndTime": 1566275032.45336, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 7}, "StartTime": 1566275032.453257}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004708522209570036, "sum": 0.004708522209570036, "min": 0.004708522209570036}}, "EndTime": 1566275032.453459, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 7}, "StartTime": 1566275032.453438}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0055916021636

[2019-08-20 04:23:58.820] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 19, "duration": 6360, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005391925841719661, "sum": 0.005391925841719661, "min": 0.005391925841719661}}, "EndTime": 1566275038.820361, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 8}, "StartTime": 1566275038.820283}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0045703682456184275, "sum": 0.0045703682456184275, "min": 0.0045703682456184275}}, "EndTime": 1566275038.820452, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 8}, "StartTime": 1566275038.820437}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005392487

[2019-08-20 04:24:11.752] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 6428, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005129939455482828, "sum": 0.005129939455482828, "min": 0.005129939455482828}}, "EndTime": 1566275051.753069, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 10}, "StartTime": 1566275051.752974}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004400380541631325, "sum": 0.004400380541631325, "min": 0.004400380541631325}}, "EndTime": 1566275051.75317, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 10}, "StartTime": 1566275051.753148}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00513069580

[2019-08-20 04:24:23.746] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6142, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004969843264201179, "sum": 0.004969843264201179, "min": 0.004969843264201179}}, "EndTime": 1566275063.746527, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1566275063.746428}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004306342939935138, "sum": 0.004306342939935138, "min": 0.004306342939935138}}, "EndTime": 1566275063.746612, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1566275063.746592}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0049706502


2019-08-20 04:24:44 Uploading - Uploading generated training model
2019-08-20 04:24:44 Completed - Training job completed
[2019-08-20 04:24:29.440] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 5687, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004912319385825689, "sum": 0.004912319385825689, "min": 0.004912319385825689}}, "EndTime": 1566275069.440808, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1566275069.440705}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004274392560199278, "sum": 0.004274392560199278, "min": 0.004274392560199278}}, "EndTime": 1566275069.440909, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 156627

Billable seconds: 160
CPU times: user 627 ms, sys: 45.4 ms, total: 672 ms
Wall time: 4min 42s


## Deploy the trained model

In [15]:
%%time 
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

----------------------------------------------------------------------------------------------------!CPU times: user 524 ms, sys: 27.6 ms, total: 551 ms
Wall time: 8min 25s



## Evaluating Your Model

In [16]:
test_x_np = test_features.astype('float32')
result = linear_predictor.predict(test_x_np[0]) 

print(result)

[label {
  key: "predicted_label"
  value {
    float32_tensor {
      values: 0.0
    }
  }
}
label {
  key: "score"
  value {
    float32_tensor {
      values: 0.001805478474125266
    }
  }
}
]


## Create Helper function for evaluation

In [17]:
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """

    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 100)]

    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch]) 
                                 for batch in prediction_batches])

    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}

In [18]:
prediction_batches = [linear_predictor.predict(batch) for batch in np.array_split(test_features.astype('float32'), 100)]

In [19]:
prediction_batches[:3]

[[label {
    key: "predicted_label"
    value {
      float32_tensor {
        values: 0.0
      }
    }
  }
  label {
    key: "score"
    value {
      float32_tensor {
        values: 0.001805478474125266
      }
    }
  }, label {
    key: "predicted_label"
    value {
      float32_tensor {
        values: 0.0
      }
    }
  }
  label {
    key: "score"
    value {
      float32_tensor {
        values: 0.00041139329550787807
      }
    }
  }, label {
    key: "predicted_label"
    value {
      float32_tensor {
        values: 0.0
      }
    }
  }
  label {
    key: "score"
    value {
      float32_tensor {
        values: 0.00041057250928133726
      }
    }
  }, label {
    key: "predicted_label"
    value {
      float32_tensor {
        values: 0.0
      }
    }
  }
  label {
    key: "score"
    value {
      float32_tensor {
        values: 0.00023044056433718652
      }
    }
  }, label {
    key: "predicted_label"
    value {
      float32_tensor {
        values: 0.

## Test Results

In [22]:
print('Metrics for simple, LinearLearner.\n')

metrics = evaluate(linear_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for simple, LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0.0               85269   33
1.0                  32  109

Recall:     0.773
Precision:  0.768
Accuracy:   0.999



## Delete the Endpoint

In [23]:
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [24]:
delete_endpoint(linear_predictor)

Deleted linear-learner-2019-08-20-04-20-12-239


## Improvement: Model Tuning

- SageMaker provides a number of ways to automatically tune a model.


## Create a LinearLearner and tune for higher precision with `binary_classifier_model_selection_criteria`  and `target_recall`

In [25]:
linear_recall = LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', 
                              target_recall=0.9)

## Train the tuned estimator

In [26]:
%%time 
linear_recall.fit(formatted_train_data)

2019-08-20 04:34:38 Starting - Starting the training job...
2019-08-20 04:34:41 Starting - Launching requested ML instances.........
2019-08-20 04:36:15 Starting - Preparing the instances for training...
2019-08-20 04:36:58 Downloading - Downloading input data...
2019-08-20 04:37:39 Training - Downloading the training image..
Docker entrypoint called with argument(s): train
[08/20/2019 04:37:54 INFO 140546649667392] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'auto', u'target_recall'

[2019-08-20 04:38:07.479] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 6647, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.01697396398668912, "sum": 0.01697396398668912, "min": 0.01697396398668912}}, "EndTime": 1566275887.479685, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1566275887.479582}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.014988560247660881, "sum": 0.014988560247660881, "min": 0.014988560247660881}}, "EndTime": 1566275887.479764, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1566275887.479746}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0169684310845993

[2019-08-20 04:38:14.019] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 6533, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.010330928617985404, "sum": 0.010330928617985404, "min": 0.010330928617985404}}, "EndTime": 1566275894.019548, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1566275894.01946}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.008765228896883864, "sum": 0.008765228896883864, "min": 0.008765228896883864}}, "EndTime": 1566275894.019625, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1566275894.019607}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.01032712537918

[2019-08-20 04:38:26.541] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 5994, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0069169620389315355, "sum": 0.0069169620389315355, "min": 0.0069169620389315355}}, "EndTime": 1566275906.54122, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1566275906.541122}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00574524320429893, "sum": 0.00574524320429893, "min": 0.00574524320429893}}, "EndTime": 1566275906.5413, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1566275906.541282}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006915968100629

[2019-08-20 04:38:38.906] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 15, "duration": 6035, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005869037068668922, "sum": 0.005869037068668922, "min": 0.005869037068668922}}, "EndTime": 1566275918.906815, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 6}, "StartTime": 1566275918.906732}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004911643637484642, "sum": 0.004911643637484642, "min": 0.004911643637484642}}, "EndTime": 1566275918.906891, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 6}, "StartTime": 1566275918.906874}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005869176341

...

[2019-08-20 04:38:56.573] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 21, "duration": 6195, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005243560808387833, "sum": 0.005243560808387833, "min": 0.005243560808387833}}, "EndTime": 1566275936.573367, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 9}, "StartTime": 1566275936.573262}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004472267618430919, "sum": 0.004472267618430919, "min": 0.004472267618430919}}, "EndTime": 1566275936.573445, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 9}, "StartTime": 1566275936.573427}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005244257314

[2019-08-20 04:39:08.793] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 25, "duration": 6389, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0050408838328404645, "sum": 0.0050408838328404645, "min": 0.0050408838328404645}}, "EndTime": 1566275948.79372, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1566275948.79362}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004347342520802464, "sum": 0.004347342520802464, "min": 0.004347342520802464}}, "EndTime": 1566275948.793806, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1566275948.793787}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005041672

[2019-08-20 04:39:14.914] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6114, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004969843264201179, "sum": 0.004969843264201179, "min": 0.004969843264201179}}, "EndTime": 1566275954.914473, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1566275954.914342}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004306342939935138, "sum": 0.004306342939935138, "min": 0.004306342939935138}}, "EndTime": 1566275954.914577, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1566275954.914555}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0049706502


2019-08-20 04:39:40 Uploading - Uploading generated training model
2019-08-20 04:39:40 Completed - Training job completed
[2019-08-20 04:39:27.682] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 6461, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0048651198454238665, "sum": 0.0048651198454238665, "min": 0.0048651198454238665}}, "EndTime": 1566275967.683094, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1566275967.682997}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004249175175650036, "sum": 0.004249175175650036, "min": 0.004249175175650036}}, "EndTime": 1566275967.683172, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 156

Billable seconds: 162
CPU times: user 691 ms, sys: 39.1 ms, total: 730 ms
Wall time: 5min 13s


## Deploy and evaluate the tuned estimator

In [27]:
%%time 
recall_predictor = linear_recall.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

---------------------------------------------------------------------------------------------------------------------------!CPU times: user 598 ms, sys: 63.2 ms, total: 661 ms
Wall time: 10min 21s


In [28]:
print('Metrics for tuned (recall), LinearLearner.\n')

metrics = evaluate(recall_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for tuned (recall), LinearLearner.

prediction (col)    0.0   1.0
actual (row)                 
0.0               81913  3389
1.0                  10   131

Recall:     0.929
Precision:  0.037
Accuracy:   0.960



## Delete the endpoint 

In [29]:
delete_endpoint(recall_predictor)

Deleted linear-learner-2019-08-20-04-34-38-216


## Improvement: Managing Class Imbalance with 
`positive_example_weight_mult`
- Create a LinearLearner with a `positive_example_weight_mult` parameter


In [30]:
linear_balanced = LinearLearner(role=role,
                                train_instance_count=1, 
                                train_instance_type='ml.c4.xlarge',
                                predictor_type='binary_classifier',
                                output_path=output_path,
                                sagemaker_session=sagemaker_session,
                                epochs=15,
                                binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                                target_recall=0.9,
                                positive_example_weight_mult='balanced')


## Train the balanced estimator


In [31]:
%%time 
linear_balanced.fit(formatted_train_data)

2019-08-20 04:50:24 Starting - Starting the training job...
2019-08-20 04:50:25 Starting - Launching requested ML instances......
2019-08-20 04:51:38 Starting - Preparing the instances for training......
2019-08-20 04:52:31 Downloading - Downloading input data...
2019-08-20 04:53:25 Training - Training image download completed. Training in progress..
Docker entrypoint called with argument(s): train
[08/20/2019 04:53:28 INFO 139783218505536] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': 

[2019-08-20 04:53:41.261] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 6516, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.4400108514526981, "sum": 0.4400108514526981, "min": 0.4400108514526981}}, "EndTime": 1566276821.261256, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1566276821.261155}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.41052518957703554, "sum": 0.41052518957703554, "min": 0.41052518957703554}}, "EndTime": 1566276821.261386, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1566276821.261365}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max

[2019-08-20 04:53:54.634] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 6837, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3764211182714108, "sum": 0.3764211182714108, "min": 0.3764211182714108}}, "EndTime": 1566276834.634185, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1566276834.634094}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3615751494211168, "sum": 0.3615751494211168, "min": 0.3615751494211168}}, "EndTime": 1566276834.634258, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1566276834.63424}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0

[2019-08-20 04:54:00.706] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6065, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3651828865549672, "sum": 0.3651828865549672, "min": 0.3651828865549672}}, "EndTime": 1566276840.706516, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1566276840.706456}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3535009060193546, "sum": 0.3535009060193546, "min": 0.3535009060193546}}, "EndTime": 1566276840.7066, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1566276840.706585}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0

[2019-08-20 04:54:12.796] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 15, "duration": 5922, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.35317101517873795, "sum": 0.35317101517873795, "min": 0.35317101517873795}}, "EndTime": 1566276852.796187, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 6}, "StartTime": 1566276852.796095}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.34528309504830057, "sum": 0.34528309504830057, "min": 0.34528309504830057}}, "EndTime": 1566276852.796279, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 6}, "StartTime": 1566276852.796259}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, 

...

[2019-08-20 04:54:31.843] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 21, "duration": 6616, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.34497469671048114, "sum": 0.34497469671048114, "min": 0.34497469671048114}}, "EndTime": 1566276871.844006, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 9}, "StartTime": 1566276871.843923}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.33982736052700024, "sum": 0.33982736052700024, "min": 0.33982736052700024}}, "EndTime": 1566276871.84409, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 9}, "StartTime": 1566276871.844074}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "

[2019-08-20 04:54:43.708] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 25, "duration": 5902, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.34214417086653975, "sum": 0.34214417086653975, "min": 0.34214417086653975}}, "EndTime": 1566276883.708982, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1566276883.708899}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3377189544601057, "sum": 0.3377189544601057, "min": 0.3377189544601057}}, "EndTime": 1566276883.709065, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1566276883.709047}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "

[2019-08-20 04:54:55.678] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 5707, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3403380548678451, "sum": 0.3403380548678451, "min": 0.3403380548678451}}, "EndTime": 1566276895.678867, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1566276895.678783}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3361356635165574, "sum": 0.3361356635165574, "min": 0.3361356635165574}}, "EndTime": 1566276895.678967, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1566276895.678947}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max

[2019-08-20 04:55:01.757] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 6072, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.33968455390355096, "sum": 0.33968455390355096, "min": 0.33968455390355096}}, "EndTime": 1566276901.75784, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1566276901.75778}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.33545819808729926, "sum": 0.33545819808729926, "min": 0.33545819808729926}}, "EndTime": 1566276901.75793, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1566276901.757915}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "


2019-08-20 04:55:14 Uploading - Uploading generated training model
2019-08-20 04:55:14 Completed - Training job completed
Billable seconds: 163
CPU times: user 683 ms, sys: 46 ms, total: 729 ms
Wall time: 5min 13s


## Deploy and evaluate the balanced estimator

In [32]:
%%time 
balanced_predictor = linear_balanced.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

--------------------------------------------------------------------------------------------------------------------------!CPU times: user 613 ms, sys: 45.6 ms, total: 659 ms
Wall time: 10min 16s


In [33]:
print('Metrics for balanced, LinearLearner.\n')

metrics = evaluate(balanced_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for balanced, LinearLearner.

prediction (col)    0.0   1.0
actual (row)                 
0.0               84277  1025
1.0                  12   129

Recall:     0.915
Precision:  0.112
Accuracy:   0.988



## Delete the endpoint 

In [34]:
delete_endpoint(balanced_predictor)

Deleted linear-learner-2019-08-20-04-50-24-392


## Train and deploy a LinearLearner with  `target_precision`

In [35]:
%%time

linear_precision = LinearLearner(role=role,
                                train_instance_count=1, 
                                train_instance_type='ml.c4.xlarge',
                                predictor_type='binary_classifier',
                                output_path=output_path,
                                sagemaker_session=sagemaker_session,
                                epochs=15,
                                binary_classifier_model_selection_criteria='recall_at_target_precision',
                                target_precision=0.9,
                                positive_example_weight_mult='balanced')


linear_precision.fit(formatted_train_data)

2019-08-20 05:06:06 Starting - Starting the training job...
2019-08-20 05:06:10 Starting - Launching requested ML instances......
2019-08-20 05:07:13 Starting - Preparing the instances for training...
2019-08-20 05:08:03 Downloading - Downloading input data...
2019-08-20 05:08:12 Training - Downloading the training image..
Docker entrypoint called with argument(s): train
[08/20/2019 05:08:43 INFO 140016668235584] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'auto', u'target_recall': u


2019-08-20 05:08:40 Training - Training image download completed. Training in progress.[2019-08-20 05:08:57.305] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7299, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.4400108514526981, "sum": 0.4400108514526981, "min": 0.4400108514526981}}, "EndTime": 1566277737.305497, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1566277737.305413}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.41052518957703554, "sum": 0.41052518957703554, "min": 0.41052518957703554}}, "EndTime": 1566277737.305573, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1566277737.305559}

#metrics {"Metr

[2019-08-20 05:09:03.701] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 6390, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.39636924766655546, "sum": 0.39636924766655546, "min": 0.39636924766655546}}, "EndTime": 1566277743.702083, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1566277743.701999}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.376585278074945, "sum": 0.376585278074945, "min": 0.376585278074945}}, "EndTime": 1566277743.702173, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1566277743.702154}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 

[2019-08-20 05:09:16.030] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6516, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3651828865549672, "sum": 0.3651828865549672, "min": 0.3651828865549672}}, "EndTime": 1566277756.031118, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1566277756.031031}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3535009060193546, "sum": 0.3535009060193546, "min": 0.3535009060193546}}, "EndTime": 1566277756.031209, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1566277756.031189}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max":

[2019-08-20 05:09:22.273] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 13, "duration": 6235, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.35806176627461034, "sum": 0.35806176627461034, "min": 0.35806176627461034}}, "EndTime": 1566277762.273193, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1566277762.273093}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.34855323484794576, "sum": 0.34855323484794576, "min": 0.34855323484794576}}, "EndTime": 1566277762.273294, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1566277762.273273}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, 

...

[2019-08-20 05:09:46.613] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 21, "duration": 6454, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.34497469671048114, "sum": 0.34497469671048114, "min": 0.34497469671048114}}, "EndTime": 1566277786.613444, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 9}, "StartTime": 1566277786.613351}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.33982736052700024, "sum": 0.33982736052700024, "min": 0.33982736052700024}}, "EndTime": 1566277786.613525, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 9}, "StartTime": 1566277786.613505}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, 

[2019-08-20 05:09:52.360] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 5740, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.343385628781726, "sum": 0.343385628781726, "min": 0.343385628781726}}, "EndTime": 1566277792.360554, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 10}, "StartTime": 1566277792.360469}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.33867450073855604, "sum": 0.33867450073855604, "min": 0.33867450073855604}}, "EndTime": 1566277792.360638, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 10}, "StartTime": 1566277792.360623}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max

[2019-08-20 05:10:04.708] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6133, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.34114156809284457, "sum": 0.34114156809284457, "min": 0.34114156809284457}}, "EndTime": 1566277804.70899, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1566277804.708896}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3368815071451005, "sum": 0.3368815071451005, "min": 0.3368815071451005}}, "EndTime": 1566277804.709082, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1566277804.709061}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "m

[2019-08-20 05:10:17.657] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 6754, "num_examples": 200, "num_bytes": 33493152}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.33968455390355096, "sum": 0.33968455390355096, "min": 0.33968455390355096}}, "EndTime": 1566277817.657651, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1566277817.657566}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.33545819808729926, "sum": 0.33545819808729926, "min": 0.33545819808729926}}, "EndTime": 1566277817.657746, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1566277817.657725}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1


2019-08-20 05:10:29 Uploading - Uploading generated training model
2019-08-20 05:10:29 Completed - Training job completed
Billable seconds: 146
CPU times: user 623 ms, sys: 53.3 ms, total: 676 ms
Wall time: 4min 42s


In [36]:
%%time 
precision_predictor = linear_precision.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

---------------------------------------------------------------------------------------------------!CPU times: user 521 ms, sys: 22.7 ms, total: 543 ms
Wall time: 8min 20s


In [37]:
print('Metrics for tuned (precision), LinearLearner.\n')

metrics = evaluate(precision_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for tuned (precision), LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0.0               85276   26
1.0                  31  110

Recall:     0.780
Precision:  0.809
Accuracy:   0.999



In [38]:
delete_endpoint(precision_predictor)

Deleted linear-learner-2019-08-20-05-06-06-496
